# iqplot

[Data set download](https://s3.amazonaws.com/bebi103.caltech.edu/data/gfmt_sleep.csv)

<hr />

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade iqplot colorcet watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"

In [2]:
import pandas as pd

import iqplot

import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

<hr>

I developed [iqplot](https://iqplot.github.io/), which generates Bokeh plots for data sets where exactly one variable is quantitative and all other variables of interest, if any, are categorical. This is where the name comes from; the first two letters of the package name are meant to indicate one (Roman number I) quantitative (Q) variable. The subclass of data sets that contain a single quantitative variable (and possibly several categorical variables) abound in the biological sciences.

There are seven types of plots that iqplot can generate. As you will see, all of these modes of plotting are meant to give a picture about how the quantitative measurements are *distributed*.

+ Box plots: `iqplot.box()`
+ Strip plots: `iqplot.strip()`
+ Spike plots: `iqplot.spike()`
+ Strip-box plots (strip and box plots overlaid): `iqplot.stripbox()`
+ Histograms: `iqplot.histogram()`
+ Strip-histogram plots (strip and histograms overlaid): `iqplot.striphistogram()`
+ ECDFs: `iqplot.ecdf()`
    
This first seven arguments are the same for all plots. They are:

- `data`: A tidy data frame
- `q`: The column of the data frame to be treated as the quantitative variable.
- `cats`: A list of columns in the data frame that are to be considered as categorical variables in the plot. If `None`, a single box, strip, histogram, or ECDF is plotted.
- `q_axis`: Along which axis, x or y that the quantitative variable varies. The default is `'x'`.
- `palette`: A list of hex colors to use for coloring the markers for each category. By default, it uses the Glasbey Category 10 color palette from [colorcet](https://colorcet.holoviz.org/).
- `order`: If specified, the ordering of the categories to use on the categorical axis and legend (if applicable). Otherwise, the order of the inputted data frame is used.
- `p`: If specified, the `bokeh.plotting.Figure` object to use for the plot. If not specified, a new figure is created.

If `data` is given as a Numpy array, it is the only required argument. If `data` is given as a Pandas DataFrame, `q` must also be supplied. All other arguments are optional and have reasonably set defaults. Any extra kwargs not in the function call signature are passed to `bokeh.plotting.figure()` when the figure is instantiated.

With this in mind, we will put iqplot to use on facial identification data set to demonstrate how we can make each of the seven kinds of plots. First, as we have become accustomed, we'll load in the data set.

In [3]:
fname = os.path.join(data_path, "gfmt_sleep.csv")
df = pd.read_csv(fname, na_values="*")
df["insomnia"] = df["sci"] <= 16
df["sleeper"] = df["insomnia"].apply(lambda x: "insomniac" if x else "normal")
df["gender"] = df["gender"].apply(lambda x: "female" if x == "f" else "male")

## All seven plots

We now make plots of the percent correct insomniacs and normal sleepers so you can see how the syntax works.

### Box plot

In [4]:
p = iqplot.box(
    data=df,
    q="percent correct",
    cats="sleeper",
)

bokeh.io.show(p)

### Strip plot

For this plot, I will add jitter, which is passed using the `spread='jitter'` kwarg.

In [5]:
p = iqplot.strip(
    data=df,
    q="percent correct",
    cats="sleeper",
    spread="jitter",
)

bokeh.io.show(p)

### Spike plot

Spike plots show the count of each value measured. They are only effective if the measurements take on discrete values.

In [6]:
p = iqplot.spike(
    data=df,
    q="percent correct",
    cats="sleeper",
)

bokeh.io.show(p)

Note that when the spikes are in the "stacked" arrangement as above (as is default), the count values cannot be ascertained, only compared. We can display the counts using the `arranged="overlay"` keyword argument.

In [7]:
p = iqplot.spike(
    data=df,
    q="percent correct",
    cats="sleeper",
    arrangement="overlay",
    style="spike-dot",
)

bokeh.io.show(p)

### Strip-box plot

For a strip-box plot, a strip plot and box plot are overlaid with reasonable defaults for the box plot to enable visualization.

In [8]:
p = iqplot.stripbox(
    data=df,
    q="percent correct",
    cats="sleeper",
    spread="jitter",
)

bokeh.io.show(p)

### Histogram

For histograms, the number of bins are automatically chosen using the [Freedman-Diaconis rule](https://en.wikipedia.org/wiki/Freedman–Diaconis_rule).

In [9]:
p = iqplot.histogram(
    data=df,
    q="percent correct",
    cats="sleeper",
)

bokeh.io.show(p)

### Strip-histogram

Strip plots may also be combined with histograms. By default, the histograms are normalized and mirrored, similar to a [violin plot](https://en.wikipedia.org/wiki/Violin_plot).

In [10]:
p = iqplot.striphistogram(
    data=df,
    q="percent correct",
    cats="sleeper",
    spread="swarm",
)

bokeh.io.show(p)

### ECDF

In [11]:
p = iqplot.ecdf(
    data=df,
    q="percent correct",
    cats="sleeper",
)

bokeh.io.show(p)

Note that the ECDFs show a clear difference. Insomniacs have a distribution that is shifted leftward. This is most clearly revealed in the ECDF.

## Customization with iqplot

You may have noticed in the above plots that I occasionally used keyword arguments beyond the seven arguments that are present in all of the function signatures. For example in the ECDF above, I used `style='staircase'`. These are plot-type-specific kwargs which enable customization.

You can find out what kwargs are available for each function by reading their doc strings, e.g., with

```python
iqplot.box?
```

of by reading [the documentation](https://iqplot.github.io/). Any kwargs not in the function call signature are passed to `bokeh.plotting.figure()` when the figure is instantiated.

In our examples of customization, we will use both gender and sleeper status as keyword arguments.

### Customizing box plots

We can also have vertical box plots using the `q_axis` kwarg.

In [12]:
p = iqplot.box(
    data=df,
    q="percent correct",
    cats=["gender", "sleeper"],
    q_axis="y",
)

bokeh.io.show(p)

We can independently specify properties of the marks using `box_kwargs`, `whisker_kwargs`, `median_kwargs`, and `outlier_kwargs`. For example, say we wanted our colors to be [Betancourt red](https://betanalpha.github.io/assets/case_studies/principled_bayesian_workflow.html#step_four:_build_a_generative_model19), and that we wanted the outliers to also be that color and use diamond glyphs.

In [13]:
p = iqplot.box(
    data=df,
    q='percent correct',
    cats=['gender', 'sleeper'],
    q_axis='y',
    whisker_caps=True,
    outlier_marker='diamond',
    box_kwargs=dict(fill_color='#7C0000'),
    whisker_kwargs=dict(line_color='#7C0000', line_width=2),
)

bokeh.io.show(p)

### Customzing strip plots

To help alleviate the overlap problem, we can make a strip plot with dash markers and add some transparency.

In [14]:
p = iqplot.strip(
    data=df,
    q='percent correct',
    cats=['gender', 'sleeper'],
    marker='dash',
    marker_kwargs=dict(alpha=0.5)
)

bokeh.io.show(p)

I prefer jittering or swarming to this. Below, I add hover tools that give more information about the respective data points in a jittered strip plot.

In [15]:
p = iqplot.strip(
    data=df,
    q="percent correct",
    cats=["gender", "sleeper"],
    spread="swarm", # use spread='jitter' to add random jitter
    tooltips=[("age", "@{age}"), ("participant number", "@{participant number}")],
)

bokeh.io.show(p)

/Users/bois/anaconda3/envs/bebi103_build/lib/python3.11/site-packages/iqplot/cat.py:1771: UserWarning: 1 data points exceed maximum height. Consider using spread='jitter' or increasing the frame height.
  warnings.warn(


Note the warning that a data point exceeds the maximum spread allowed in a swarm plot such that data points overlap. For this reason, I often prefer jitter plots to swarm plots. In this particular case, because of the discrete values taken by the percent correct, a swarm plot is more effective at showing all data.

### Customizing histograms

We can plot normalized histograms using the `density` kwarg, and we'll make the plot a little wider to support the legend.

In [16]:
# Plot the histogram
p = iqplot.histogram(
    data=df,
    q='percent correct',
    cats=['gender', 'sleeper'],
    density=True,
    frame_width=525,
)

bokeh.io.show(p)

#### Customizing ECDFs

Instead of plotting a separate ECDF for each category, we can put all of the categories together on one ECDF and color the points by the categorical variable by using the `kind='colored'` kwarg. Note that if we do this, we can only have the "dots" style ECDF, not the formal staircase.

In [17]:
p = iqplot.ecdf(
    data=df,
    q="percent correct",
    cats=["gender", "sleeper"],
    kind="colored",
)

bokeh.io.show(p)

In general, for customization, you should check [the documentation](https://iqplot.github.io/) to see what is available.

## Computing environment

In [18]:
%load_ext watermark
%watermark -v -p pandas,bokeh,iqplot,jupyterlab

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

pandas    : 2.0.3
bokeh     : 3.2.1
iqplot    : 0.3.5
jupyterlab: 4.0.6

